logica: miro una fecha y un bidder, miro sus competidores, busco dias similares y comportamiento de sus competidores. Despues miro M muestra boostratps, como escenarios y veo el equilibrio, de como juegan sus competidores. Después miro, en cada una de esas M muestras bootstratps para ese dia hallo la derivada, despues hago el promedio sobre todas las muestras bootstratps. Depues hago la suma sobre todas las horas y eso va a ser el phi, con los otros terminos

In [1]:
#librerias
import pandas as pd
import numpy as np
from tqdm import tqdm
from joblib import Parallel, delayed
from scipy.stats import norm
from tqdm.auto import tqdm
from tqdm_joblib import tqdm_joblib

In [2]:
#cargar archivos
df_final = pd.read_csv("../datos/procesado/df_final.csv")
df_transado  =pd.read_csv("../datos/procesado/df_despacho_agg.csv")

In [3]:
df_transado = df_transado.rename(columns={'daily_eq_demand': 'demanda'})
df_final= df_final[df_final['cantidad']!=0]


In [4]:
# ----------------------------------------------------
# FUNCIONES AUXILIARES
# ----------------------------------------------------

def get_cluster_it(df, fecha, firma):
    """Devuelve el cluster correspondiente a una firma i en una FechaHora t."""
    row = df.loc[
        (df['FechaHora'] == fecha) & (df['CodigoPlanta'] == firma),
        'cluster'
    ]
    return row.iloc[0] if not row.empty else np.nan


def get_competitors(df, fecha, firma):
    """Devuelve los competidores (otros CodigoPlanta) presentes en la misma FechaHora."""
    df_day = df[df['FechaHora'] == fecha]
    competitors = df_day.loc[df_day['CodigoPlanta'] != firma, 'CodigoPlanta'].unique().tolist()
    return competitors


def get_similar_days_by_cluster(df, fecha, firma, max_obs=20):
    """
    Busca días similares según el cluster del competidor k != i.
    Para cada competidor en t, extrae observaciones con el mismo cluster que i.
    """
    cluster_it = get_cluster_it(df, fecha, firma)
    competitors = get_competitors(df, fecha, firma)
    similar_obs = []

    for comp in competitors:
        df_comp_similar = df[
            (df['CodigoPlanta'] == comp) & (df['cluster'] == cluster_it)
        ].copy()

        if len(df_comp_similar) > max_obs:
            df_comp_similar = df_comp_similar.sample(n=max_obs, random_state=42)

        df_comp_similar['competidor_de'] = firma
        df_comp_similar['fecha_base'] = fecha
        similar_obs.append(df_comp_similar)

    return pd.concat(similar_obs, ignore_index=True) if similar_obs else pd.DataFrame()


def bootstrap_by_planta(df, M, seed=None):
    """Genera M muestras bootstrap independientes seleccionando 1 observación por planta."""
    if seed is not None:
        np.random.seed(seed)

    plantas = df['CodigoPlanta'].unique()
    bootstrap_samples = []

    for m in range(M):
        muestras = []
        for p in plantas:
            df_p = df[df['CodigoPlanta'] == p]
            if len(df_p) == 0:
                continue
            muestra = df_p.sample(1, replace=True)
            muestra['bootstrap_id'] = m + 1
            muestras.append(muestra)
        sample_df = pd.concat(muestras).reset_index(drop=True)
        bootstrap_samples.append(sample_df)

    return bootstrap_samples


def compute_equilibrium(df_offers, df_transado_date):
    """
    Encuentra el precio y cantidad de equilibrio (p*, q*) para un conjunto de ofertas.
    df_transado_date debe contener la demanda para la FechaHora actual.
    """
    fecha = df_offers['fecha_base'].iloc[0]
    demanda_row = df_transado_date.loc[df_transado_date['FechaHora'] == fecha, 'demanda']

    if demanda_row.empty:
        return np.nan, np.nan

    demand = demanda_row.iloc[0]
    df_sorted = df_offers.sort_values('precio').copy()
    df_sorted['acum'] = df_sorted['cantidad'].cumsum()

    clearing_offers = df_sorted[df_sorted['acum'] >= demand]
    if clearing_offers.empty:
        return np.nan, demand

    p_star = clearing_offers.iloc[0]['precio']
    q_star = demand
    return p_star, q_star

In [5]:
# ----------------------------------------------------
# KERNELS
# ----------------------------------------------------
# --- 1. Definiciones del Kernel ---

def gaussian_kernel(u):
    """Kernel Gaussiano estándar (PDF de N(0, 1))."""
    return norm.pdf(u)

def gaussian_kernel_prime(u):
    """Derivada del Kernel Gaussiano: κ'(u) = -u * κ(u)."""
    return -u * gaussian_kernel(u)   #ya es como si tuviera el negativo, u lo reescribo como  pht-pkt por justificacion

# --- 2. Estimación de la Demanda Residual (RD(p)) ---

# Asumo que tienes una función para obtener pos_it, o que se añade como argumento

def kernel_expectation(df, p_ht, D, gamma, firma_i):
    """
    Calcula la Demanda Residual Neta:
       RD(p) = D - sum_k g_k * K((b_k - p)/gamma)
    para la firma i.
    """
    pos_it = 0  # si luego quieres incluir posición pre-asignada

    if df.empty:
        return D - pos_it  # si no hay rivales, demanda residual = demanda total

    # Argumento del kernel
    u_others = (df["precio"] - p_ht) / gamma

    # Kernel gaussiano
    weights_others = gaussian_kernel(u_others)

    # --- AQUÍ LA CORRECCIÓN CLAVE ---
    # Es una SUMA ponderada, NO un promedio
    S_minus_i = (df["cantidad"] * weights_others).sum()

    # Demanda residual bruta
    RD_p = D - S_minus_i

    # Restar posición pre-existente
    RD_neta = RD_p - pos_it

    return RD_neta


# --- 3. Estimación de la Derivada de la Demanda Residual (RD'(p)) ---

def kernel_derivative(df, p_ht, gamma, firma_i):
    """
    Calcula la derivada de la Demanda Residual:
        RD'(p) = sum_{k ≠ i} g_k * (1/gamma) * K'((b_k - p)/gamma)
    """
    if df.empty:
        return 0.0

    # Coincidir EXACTAMENTE con tu fórmula: (b_k - p_ht)/gamma
    u_others = (df["precio"] - p_ht) / gamma

    # K'(u) = -u K(u)
    weights_prime = gaussian_kernel_prime(u_others)

    # SUMA ponderada (no promedio)
    dQ = (df["cantidad"] * weights_prime).sum() / gamma

    return dQ


In [85]:
# ----------------------------------------------------
# PROMEDIO SOBRE BOOTSTRAPS
# ----------------------------------------------------

def average_numerador_denom(df_bootstrap_list, df_transado, df_full, fecha_hora_i_t, gamma, firma):
    """
    Promedia el numerador y denominador kernelizados sobre varias muestras bootstrap.
    Devuelve el promedio (numer, denom) y el último p*, q* observados.
    """
    if not df_bootstrap_list:
        return np.nan, np.nan, None, None

    muestras=len(df_bootstrap_list) 
    

    numeradores = []
    denominadores = []

    # Precomputar para cada bootstrap (vectorizado en el sentido de loops limpios)
    for df_sim in df_bootstrap_list:
        
        firma=df_full[
            (df_full['CodigoPlanta'] == firma) &
            (df_full['FechaHora'] == fecha_hora_i_t)]
        
        #todos lo bidders incluyendo i
        df_bidders = pd.concat([df_sim, firma], ignore_index=True)

        p_ht_bs,  q_ht_bs = compute_equilibrium(df_bidders, df_transado)
        D= df_transado[(df_transado['FechaHora'] == fecha_hora_i_t)]['demanda']

        #exluir a bidder i
        # Numerador ≈ E[-it][Q | s, p = b_it]
        numer = kernel_expectation(df_sim, p_ht_bs, D, gamma, firma)

        # Denominador ≈ E[-it][dQ/db | s, p = b_it]
        denom = kernel_derivative(df_sim, p_ht_bs, gamma, firma)

        numeradores.append(numer)
        denominadores.append(denom)

    # Convertir a vectores numpy
    numeradores = np.array(numeradores)
    denominadores = np.array(denominadores)

    # Promedios
    avg_numer = numeradores.mean()
    avg_denom = denominadores.mean()

    return avg_numer, avg_denom, numeradores, denominadores, p_ht_bs,  q_ht_bs
   


In [86]:

# ----------------------------------------------------
# FUNCIÓN PRINCIPAL
# ----------------------------------------------------

# Función auxiliar para el trabajo de una sola fila
def process_row(row, df_full, gamma, M):
    fecha = row.FechaHora          # <-- antes row['FechaHora']
    firma = row.CodigoPlanta       # <-- antes row['CodigoPlanta']
    
    # 1) Días similares
    df_similares = get_similar_days_by_cluster(df_full, fecha, firma)
    
    
    # 2) Bootstraps
    df_bootstrap_list = bootstrap_by_planta(df_similares, M, seed=123)

    # 3) RD(b_it) y RD'(b_it) promedio
    avg_numer, avg_denom, numeradores, denominadores, p_ht_bs,  q_ht_bs = average_numerador_denom(
        df_bootstrap_list,
        df_transado,
        df_full,
        fecha,
        gamma,
        firma
    )
    
    return df_similares, df_bootstrap_list, avg_numer, avg_denom, p_ht_bs,  q_ht_bs


def calcular_avg_Q_y_dQdb_parallel(df, gamma, M, seed=123, n_jobs=-1):

    with tqdm_joblib(tqdm(total=len(df), desc="Calculando", unit="fila")):
        results = Parallel(n_jobs=n_jobs, backend='loky')(
            delayed(process_row)(row, df, gamma, M) 
            for row in df.itertuples(index=False)  # iteramos como tu querías
        )

    # Separar resultados
    EQ_results = [res[0] for res in results]
    dQdb_results = [res[1] for res in results]

    # Agregar al DataFrame
    df['EQpos'] = EQ_results
    df['EdQb'] = dQdb_results
    
    return df

## Test

In [97]:
df_transado=df_transado[df_transado['FechaHora']=='2025-05-25 23:00:00']
df_transado.loc[1319, 'demanda'] = 180000


In [98]:
row=df_final.iloc[0]
df_full = df_final[df_final['CodigoPlanta'].isin(['JAGS', 'ZPA2', 'ZPA2', 'TEC1'])]
gamma=10
M=1

In [99]:
row

FechaHora       2025-05-25 23:00:00
CodigoPlanta                   TEC1
precio                      1486.46
cantidad                   213000.0
Fecha                    2025-05-25
cluster                           5
Name: 0, dtype: object

In [100]:
df_similares, df_bootstrap_list, avg_numer, avg_denom, p_ht_bs,  q_ht_bs= process_row(row, df_full, gamma, M)

In [101]:
df_transado

,FechaHora,demanda
1319,2025-05-25 23:00:00,180000


In [108]:
df_bootstrap_list[0]

,FechaHora,CodigoPlanta,precio,cantidad,Fecha,cluster,competidor_de,fecha_base,bootstrap_id
0,2025-05-23 17:00:00,JAGS,102.64,170000.0,2025-05-23,5,TEC1,2025-05-25 23:00:00,1
1,2025-05-25 07:00:00,ZPA2,373.39,36000.0,2025-05-25,5,TEC1,2025-05-25 23:00:00,1


In [105]:
p_ht_bs

np.float64(373.39)

In [106]:
q_ht_bs

np.int64(180000)

In [103]:
avg_numer

np.float64(165638.07790554842)

In [104]:
avg_denom

np.float64(1.2103779955053954e-154)

## Aplicar a toda la base

In [ ]:
df_resultado = calcular_avg_Q_y_dQdb_parallel(
    df=df_final, 
    gamma=10, 
    M=1, 
    n_jobs=-1
)

## Guardar resultados 

In [ ]:

# Crear carpeta principal
output_dir = "../results"
os.makedirs(output_dir, exist_ok=True)

# Calcular número de fechas únicas
n_fechas = df_filtrado["Fecha"].nunique()

# Crear subcarpeta con el número de fechas únicas
sub_dir = os.path.join(output_dir, f"fechas_{n_fechas}")
os.makedirs(sub_dir, exist_ok=True)

print(f"Carpeta creada: {sub_dir}")